In [6]:
import scipy.misc as misc
import os
import numpy as np
import sys
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
sys.path.append(os.path.expanduser('~/Documents/zhenlinx/code/2017Summer/metrics'))
from eval_segm import *
from seg_metric import SegMetric

# Evaludate results of DeepLab on Norfolk building images

In [8]:
city_name = 'Norfolk'
ind = 1
truthDir = os.path.expanduser("~/Documents/data/building/{}".format(city_name, city_name, ind ))
learning_rates = [1e-3]
batch_sizes = [20]
num_epochs = 30
input_size = '128,128'
decay_step = 10 # in epochs
decay_rate = 0.1
IMAGE_SIZE = 128
weight_decays = [0.0005, 0.0001]
imageFileName = ["{}_{:0>2}{}".format(city_name, k, "{}") for k in range(1,3) ]

for learning_rate in learning_rates:
    for batch_size in batch_sizes:
        for weight_decay in weight_decays:
#             validDir = os.path.join(os.path.realpath('../snapshots_building/train_with_pretrained_model'), 'batchsize{}_learningRate_{:.0e}_weight_decay_{}'.format(batch_size, learning_rate, weight_decay),'images') # older format           
            validDir = os.path.join(os.path.realpath('../snapshots_building/train_with_pretrained_model'), '{}_{:0>2}_batchsize{}_learningRate_{:.0e}_L2weight_{}_decayStep_{:d}_decayRate{}'.format(city_name, ind, batch_size, learning_rate, weight_decay, decay_step, decay_rate),'images')
            seg_metric = SegMetric(1)
            plt.figure("PR Curve batchsize{} LR{:.0e} weight_decay{}".format(batch_size, learning_rate, weight_decay))
            print("Evaluate batchsize{} LR{:.0e} weight_decay{}".format(batch_size, learning_rate, weight_decay))
            for i in range(0, 2):
                image = misc.imread(os.path.join(truthDir,imageFileName[i].format('_RGB.png')))
                truth = (misc.imread(os.path.join(truthDir,imageFileName[i].format('_truth.png')))/255).astype(np.uint8)
                valid_pmap = misc.imread(os.path.join(validDir,imageFileName[i].format('_valid_pmap.tif')))
                pred_binary = misc.imread(os.path.join(validDir,imageFileName[i].format('_valid_pred.png')))
                pred_binary = (valid_pmap>0.5).astype(np.uint8)
                
                print(np.unique(truth))
#                 print(f1_score(truth, pred_binary))
                
                seg_metric.add_image_pair(pred_binary, truth)
                print("mean_IoU of image {}: {:.4f}".format(imageFileName[i][0:-2],mean_IU(pred_binary, truth)))            
                      
                # plot PR curve
                plt.figure("PR Curve batchsize{} LR{:.0e} weight_decay{}".format(batch_size, learning_rate, weight_decay))
                precision, recall, thresholds = precision_recall_curve(truth.flatten(), valid_pmap.flatten(), 1)
                plt.plot(recall, precision, lw=2,
                         label=imageFileName[i][0:-2])
                plt.legend()
                plt.xlabel('Recall')
                plt.ylabel('Precision')
                plt.ylim([0.0, 1.05])
                plt.xlim([0.0, 1.0])
                plt.title('Precision-Recal')
                
            print("Overal mean IoU of batchsize{} LR{:.0e}: {:.4f}".format(batch_size, learning_rate, seg_metric.mean_IU()))

Evaluate batchsize20 LR1e-03 weight_decay0.0005
mean_IoU of image Norfolk_01: 0.6257
[0 1]


ValueError: Target is multilabel-indicator but average='binary'. Please choose another average setting.

## visualize results in patches

In [ ]:
from sklearn.metrics import precision_recall_curve
from scipy.ndimage.measurements import *
from skimage import measure

# image files
city_name = 'Norfolk'
ind = 1
truthDir = os.path.expanduser("~/Documents/data/building/{}".format(city_name, city_name, ind ))
imageFileName = ["{}_{:0>2}{}".format(city_name, k, "{}") for k in range(1,3) ]

learning_rates = [1e-3]
batch_sizes = [20]
num_epochs = 30
input_size = '128,128'
decay_step = 10 # in epochs
decay_rate = 0.1
IMAGE_SIZE = 128
weight_decays = [0.0005, 0.0001]
